In [ ]:
# Description: This notebook creates a dataset from the given list of URLs
# https://www.scrapingbee.com/blog/crawling-python/

# Importing the required libraries
import requests
import logging
from urllib.parse import urljoin
import requests
from bs4 import BeautifulSoup

In [ ]:
# Setting the logging level
logging.basicConfig(
    format='%(asctime)s %(levelname)s:%(message)s',
    level=logging.INFO)

# Creating a class for crawling the given URLs
class Crawler:
    # Initializing the class
    def __init__(self, urls=[]):
        self.visited_urls = []
        self.urls_to_visit = urls
    # Downloading the URL
    def download_url(self, url):
        return requests.get(url).text
    # Getting the linked URLs
    def get_linked_urls(self, url, html):
        soup = BeautifulSoup(html, 'html.parser')
        for link in soup.find_all('a'):
            path = link.get('href')
            if path and path.startswith('/'):
                path = urljoin(url, path)
            yield path
    # Adding the URL to visit
    def add_url_to_visit(self, url):
        if url not in self.visited_urls and url not in self.urls_to_visit:
            self.urls_to_visit.append(url)
    # Crawling the URL
    def crawl(self, url):
        html = self.download_url(url)
        for url in self.get_linked_urls(url, html):
            self.add_url_to_visit(url)
    # Running the crawler
    def run(self):
        # Crawl until there are no more URLs to visit
        while self.urls_to_visit:
            url = self.urls_to_visit.pop(0)
            logging.info(f'Crawling: {url}')
            try:
                self.crawl(url)
            except Exception:
                logging.exception(f'Failed to crawl: {url}')
            finally:
                self.visited_urls.append(url)

if __name__ == '__main__':
    Crawler(urls=['https://forums.gta5-mods.com/', 'https://forum.cfx.re/']).run()